In [1]:
# Это нужно, чтоб увидеть NEURON
import sys
sys.path.append("/home/leon/nrn/build/lib/python")

import os
os.environ["NAVIS_SKIP_CMTK"] = "1"

os.environ["PATH"] = ":".join(
    p for p in os.environ["PATH"].split(":")
    if p != "/root/.cargo/bin"
)

import json
import pandas as pd
import pymaid


# import numpy as np
# import pandas as pd
# import networkx as nx
# from collections import deque
# from neuron import h, coreneuron
# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# from mpl_toolkits.mplot3d import Axes3D
# import plotly.graph_objects as go
# import plotly.express as px

os.chdir("../..")

# # Включаем CoreNEURON
# # coreneuron.enable = True
# # coreneuron.gpu = True
# # coreneuron.verbose = 1
# # coreneuron.num_gpus = 2
# h.nrnmpi_init()

# h.load_file('stdrun.hoc')

from scripts.NEURON_Sim_Wrapper import Network

# Get ALL 3k Neurons and Params

TODO...

Totaly, overall network will have corresponding parameters:

```
soma_params = {
    'L': 20,            # длина
    'diam': 20,         # диаметр
    'Ra': 100,          # аксиальное сопротивление
    'cm': 1,            # емкость мембраны
    
    'gnabar_hh': 0.12,  # проводимость натриевых каналов
    'gkbar_hh': 0.036,  # проводимость калиевых каналов
    'gl_hh': 0.0003,    # утечка (leak) проводимость
    'el_hh': -54.3      # обратный потенциал утечки
}


neurite_params = {
    'L': 5.0,           # µm (длина одного стандартного компартмента)
    'diam': 1.0,        # µm (компромиссный диаметр для аксона/дендрита)
    'Ra': 100.0,        # Ω·cm (типичное значение для членистоногих/общая практика)
    'cm': 1.0,          # µF/cm^2 (станд. мембранная ёмкость)
    

    'gnabar_hh': OPTIMIZED,  # S/cm^2  проводимость натриевых каналов
    'gkbar_hh': OPTIMIZED,   # S/cm^2  проводимость калиевых каналов
    'gl_hh': OPTIMIZED,      # S/cm^2  утечка (leak) проводимость
    'el_hh': -70.0           # mV      обратный потенциал утечки
}


syn_params = {
    'tau1': 0.5,        # время нарастания
    'tau2': 2.0,        # время спада
    'e': 0.0            # обратный потенциал (возбуждающий)
}


netcon_params = {
    'threshold': 0.0,    # порог (мВ) *это просто чтоб показать работоспособность
    'weight': 0.5,       # вес синапса (микросименсы)
    'delay': 3.0         # задержка (мс)
}

```

In [2]:
catmaid_url = 'https://l1em.catmaid.virtualflybrain.org'
http_user = None
http_password = None
project_id = 1

rm = pymaid.CatmaidInstance(catmaid_url, http_user, http_password, project_id)

neuron_list = pymaid.get_skids_by_annotation('mw brain and inputs')
neuron_list = list(map(str, neuron_list))

len(neuron_list)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


3016

In [3]:
# Get the neurite params for the ones
with open("./Datasets/Generated/Optimized_Neural_Params(95).json", "r") as f:
    data = json.load(f)

rows = []
for neuron_id, content in data["results"].items():
    p = content["best_params"]
    rows.append({
        "neuron_id": int(neuron_id),
        "gnabar_hh": p["dend_gnabar_hh"],
        "gkbar_hh": p["dend_gkbar_hh"],
        "gl_hh": p["dend_gl_hh"]
    })

neurites_params = pd.DataFrame(rows)

# Cast to string
neurites_params["neuron_id"] = neurites_params["neuron_id"].astype(str)

# Get only needed 20 neurons
neurites_params = neurites_params[neurites_params["neuron_id"].isin(neuron_list)]



# Set the rest of params manually
neurites_params['L'] = 5.0
neurites_params['diam'] = 1.0
neurites_params['Ra'] = 100.0
neurites_params['cm'] = 1.0
neurites_params['el_hh'] = -70.0



neurites_params

,neuron_id,gnabar_hh,gkbar_hh,gl_hh,L,diam,Ra,cm,el_hh
0,15994926,0.011470,0.006765,0.000054,5.0,1.0,100.0,1.0,-70.0
1,10934438,0.052295,0.044256,0.000327,5.0,1.0,100.0,1.0,-70.0
2,15690392,0.063497,0.058354,0.000117,5.0,1.0,100.0,1.0,-70.0
3,12367135,0.021482,0.010021,0.000700,5.0,1.0,100.0,1.0,-70.0
5,9574259,0.023199,0.012481,0.000124,5.0,1.0,100.0,1.0,-70.0
...,...,...,...,...,...,...,...,...,...
3806,16594752,0.124621,0.115972,0.002695,5.0,1.0,100.0,1.0,-70.0
3807,17913199,0.027841,0.049669,0.000052,5.0,1.0,100.0,1.0,-70.0
3808,17360010,0.029182,0.023350,0.000116,5.0,1.0,100.0,1.0,-70.0
3809,13081886,0.019758,0.007337,0.000235,5.0,1.0,100.0,1.0,-70.0


# Build the Network

In [ ]:
soma_params = {
    'L': 20,            # длина
    'diam': 20,         # диаметр
    'Ra': 100,          # аксиальное сопротивление
    'cm': 1,            # емкость мембраны
    
    'gnabar_hh': 0.12,  # проводимость натриевых каналов
    'gkbar_hh': 0.036,  # проводимость калиевых каналов
    'gl_hh': 0.0003,    # утечка (leak) проводимость
    'el_hh': -54.3      # обратный потенциал утечки
}

syn_params = {
    'tau1': 0.5,        # время нарастания
    'tau2': 2.0,        # время спада
    'e': 0.0             # обратный потенциал (возбуждающий)
}

netcon_params = {
    'threshold': 0.0,  # порог (мВ) *это просто чтоб показать работоспособность
    'weight': 0.5,       # вес синапса (микросименсы)
    'delay': 3.0         # задержка (мс)
}


net = Network(neuron_list)
net.load_graphs(allow_tqdm=True)
net.build_sections(soma_mechanism='hh',
                   soma_params=soma_params,
                   dendrite_mechanism='hh',
                   dendrite_params=neurites_params, 
                  allow_tqdm=True)
net.connect_morphology(allow_tqdm=True)
net.build_synapses(synapse_params=syn_params, netcon_params=netcon_params, allow_tqdm=True)

loading graphs:   0%|          | 0/3016 [00:00<?, ?it/s]

building sections:   0%|          | 0/3016 [00:00<?, ?it/s]

connecting morphology:   0%|          | 0/3016 [00:00<?, ?it/s]

building synapses:   0%|          | 0/3016 [00:00<?, ?it/s]

# Run the simulation

In [ ]:
# Configure the simulation
net.setup_recording()
net.setup_stimulus(neurons=['7055857', '1805418'], start_time=10, duration=10, amplitude=0.7)

# запускаем симуляцию
t, voltages = net.run(duration=50)

net.analyze(t, voltages)

net.reset()

In [ ]:
net.plot_results(t, voltages, interactive=True)